In [1]:
import pandas as pd
import pyodbc
username = 'vigrose'
password_aims = 'DQA8K@T7pXEqEt'
password_edw = 'Hufflepuff10946'

In [3]:
scraped_old = pd.read_csv('../../Data/SRS/SRS_Scrape_2021-03-19.csv')
# scraped = pd.read_csv('../../Data/SRS/SRS_Scrape_2021-06-03.csv')

In [6]:
scraped_old.columns

Index(['AAMC ID', 'Birth Country', 'Birth County', 'Birth State',
       'Citizen Country', 'Curr Class Level', 'Curr Class Level Eff Date',
       'Curr Exp Grad Date', 'Curr Med School Campus', 'Curr Status',
       ...
       'Unnamed: 41', 'Unnamed: 45', 'Unnamed: 49', 'Unnamed: 53',
       'Unnamed: 57', 'Unnamed: 61', 'Unnamed: 65', 'Unnamed: 69',
       'Unnamed: 85', 'Visa Desc'],
      dtype='object', length=118)

In [8]:
scraped_old[scraped_old['Curr Status']=='Adv-Std From Dental']

,AAMC ID,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,Unnamed: 41,Unnamed: 45,Unnamed: 49,Unnamed: 53,Unnamed: 57,Unnamed: 61,Unnamed: 65,Unnamed: 69,Unnamed: 85,Visa Desc
3117,14855538,United States of America (the),Marion,Ohio,United States of America (the),4,07/01/2020,05/01/2021,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3174,15368442,United States of America (the),Middlesex,Massachusetts,United States of America (the),2,07/13/2020,05/01/2023,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,15068143,United States of America (the),Orange,California,United States of America (the),3,07/01/2020,05/01/2022,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3318,15068132,United States of America (the),Sedgwick,Kansas,United States of America (the),3,07/01/2020,05/01/2022,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3403,14551645,United States of America (the),Fairfax City,Virginia,United States of America (the),2,07/13/2020,05/01/2023,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90575,15290510,China,NaN,Foreign,United States of America (the),1,08/03/2020,05/31/2024,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90578,13199140,Colombia,NaN,Foreign,United States of America (the),3,06/09/2020,05/31/2022,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90620,15102475,United States of America (the),Montgomery,Maryland,United States of America (the),2,09/02/2020,05/31/2023,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90624,14876454,United States of America (the),Philadelphia,Pennsylvania,United States of America (the),3,06/09/2020,05/31/2022,NaN,Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [ ]:
query_5 = \
    f'''
    SELECT 
    E.KEY_TYPE_VAL AS AAMC_ID,
    E.ENTITY_ID
    FROM informix.ENTITY_KEY_ET E
    WHERE
    E.KEY_TYPE = 'AMC'
    ;
    '''
active_aamc = pd.read_sql(con=informix, sql=query_5)
active_aamc.head()

In [ ]:
no_aamc_id = scraped_old[scraped_old['AAMC ID'].isin(active_aamc.aamc_id)==False].drop_duplicates('AAMC ID')

In [ ]:
query_3 = \
    f'''
    SELECT 
    P.ENTITY_ID,
    P.GENDER,
    P.BIRTH_DT,
    P.BIRTH_STATE_CD,
    P.ETHNICITY,
    P.BIRTH_CITY_CD,
    N.FIRST_NM,
    N.MIDDLE_NM,
    N.LAST_NM,
    S.STUD_ID,
    S.SCHOOL_ID,
    S.EDU_STS,
    S.GRAD_YR,
    S.EDU_PREF_CD,
    S.DEGREE_CD
    FROM
    informix.PERSON_NAME_ET N, informix.PERSON_ET P, informix.SCHOOL_ATT_ST S
    WHERE
    P.ENTITY_ID = N.ENTITY_ID
    AND
    P.CURR_PERSON_TYPE = 'S'
    AND
    N.END_DT IS NULL
    AND
    N.NAME_TYPE='LN'
    AND
    P.ENTITY_ID = S.STUD_ID
    ;
    '''
all_students = pd.read_sql(con=informix, sql=query_3)
all_students.head()

In [ ]:
def clean_missing(missing):
    missing = missing.fillna('None')
    missing['last_nm'] = [x.split(', ')[0] for x in missing.Name]
    missing['first_middle'] = [x.split(', ')[1] for x in missing.Name]
    missing['birth_state_cd'] = [us_state_abbrev[x] if x in us_state_abbrev.keys() else x for x in missing['Birth State']]
    missing['birth'] = pd.to_datetime(missing['Date of Birth'])
    return(missing)

In [ ]:
def get_birth(all_students):
    with_birth = all_students[all_students.birth_dt!='None']
    with_birth['birth'] = pd.to_datetime([str(x).replace(' ','') for x in with_birth.birth_dt])
    return(with_birth)

In [ ]:
all_students.birth_dt

In [ ]:
no_id = clean_missing(no_aamc_id)

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
query_6 = \
    f'''
    SELECT 
    E.KEY_TYPE_VAL AS ME,
    E.ENTITY_ID
    FROM informix.ENTITY_KEY_ET E, informix.PERSON_ET P
    WHERE
    E.KEY_TYPE = 'ME'
    AND
    P.ENTITY_ID = E.ENTITY_ID
    ;
    '''
me_entity = pd.read_sql(con=informix, sql=query_6)
me_entity.head()

In [ ]:
query_7 = \
    f'''
    SELECT 
    E.ENTITY_ID,
    E.CATEGORY_CODE,
    E.BEGIN_DT,
    E.END_DT
    FROM informix.ENTITY_CAT_CT E
    WHERE
    E.CATEGORY_CODE = 'STU-AFFIL'
    AND
    E.END_DT IS NULL
    ;
    '''
STU_AFFIL = pd.read_sql(con=informix, sql=query_7)
STU_AFFIL.head()

In [ ]:
active_aamc['aamc_id'] = active_aamc.aamc_id.astype(int)

In [ ]:
def merge_name(all_students):
    first_middles = []
    for row in all_students.itertuples():
        first = row.first_nm.strip()
        if row.middle_nm != 'None':
            middle = ' '+ row.middle_nm.strip()
        else:
            middle = ''
        first_middle = first + middle
        first_middles.append(first_middle)
    all_students['first_middle'] = first_middles
    return(all_students)

In [ ]:
all_students['last_nm'] = [x.strip() for x in all_students.last_nm]
all_students = merge_name(all_students)

In [ ]:
no_id['birth_dt'] = pd.to_datetime(no_id['Date of Birth'])

In [ ]:
with_birth = all_students[all_students.birth_dt!='None']
with_birth['birth_dt'] = pd.to_datetime(with_birth.birth_dt)

In [ ]:
no_id['Name']

In [ ]:
on_last = pd.merge(all_students, no_id, left_on = ['last_nm', 'gender'], right_on = ['last_nm', 'Sex']).drop_duplicates('AAMC ID')
on_first = pd.merge(all_students, no_id, left_on = ['first_middle', 'gender'], right_on = ['first_middle', 'Sex']).drop_duplicates('AAMC ID')

In [ ]:
so_far = list(on_last['AAMC ID']) + list(on_first['AAMC ID'])
check_12 = no_id[no_id['AAMC ID'].isin(so_far)==False]

In [ ]:
check_12[['Name','birth_dt', 'birth_state_cd']][20:40]

In [ ]:
all_students[all_students.last_nm=='Yepes'][['birth_dt', 'birth_state_cd','first_nm','last_nm']]

In [ ]:
all_students[all_students.first_nm=='Amy'][['birth_dt', 'birth_state_cd','last_nm']]

In [ ]:
all_students['first_nm'] = [x.strip() for x in all_students.first_nm]

In [ ]:
pd.merge(with_birth, no_id, left_on = ['birth_dt','last_nm', 'gender'], right_on = ['birth_dt','last_nm', 'Sex'])

In [ ]:
pd.merge(with_birth, no_id, left_on = ['birth_dt','last_nm', 'first_middle','gender', 'birth_state_cd'], right_on = ['birth_dt','last_nm', 'first_middle','Sex', 'birth_state_cd'])

In [ ]:
boop = bop.drop_duplicates(['entity_id', 'aamc_id'])
boop = boop[boop.end_dt=='None']
match = boop.drop_duplicates('aamc_id',keep=False).sort_values('grad_yr').drop_duplicates('aamc_id', keep='last')

In [ ]:
still_missing = missing[missing.aamc_id.isin(matched.aamc_id)==False]
stateless_match = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])

In [ ]:
add_match = pd.merge(person_students_3, still_missing, left_on = ['birth','last_nm', 'gender'], right_on = ['birth','last_nm', 'Sex']).drop_duplicates('aamc_id')
boopa = still_missing[still_missing.aamc_id.isin(add_match.aamc_id)==False]
no_birth_match = pd.merge(without_birth, boopa, left_on = ['first_middle','last_nm', 'gender'], right_on = ['first_middle','last_nm', 'Sex'])[['Name','first_nm','last_nm','birth_state_cd_x','birth_state_cd_y']]

In [ ]:
foreign = missing[missing['Birth State']=='Foreign']
foreign_match = pd.merge(with_birth, foreign, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])
foreign_match = foreign_match[foreign_match.end_dt=='None']

In [ ]:
still_missing = missing[missing.aamc_id.isin(matched.aamc_id)==False]
stateless_match = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])

In [ ]:
stateless_match = stateless_match[stateless_match.end_dt=='None']

In [ ]:
weird_matches = pd.merge(with_birth, still_missing, left_on = ['first_middle','last_nm', 'gender','birth'], right_on = ['first_middle','last_nm', 'Sex', 'birth']).sort_values(['end_dt', 'grad_yr']).drop_duplicates('aamc_id', keep='last')

In [ ]:
no_first = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'gender'], right_on = ['birth','last_nm', 'Sex'])
keeps =[]
for row in no_first.itertuples():
    keep=False
    if row.first_nm.replace(' ','') in row.Name:
        keep=True
    keeps.append(keep)
no_first['KEEP']=keeps
no_first = no_first[no_first.KEEP==True].drop(columns=['KEEP'])

In [ ]:
matched = pd.concat([match, foreign_match, stateless_match, weird_matches, no_first])
matched = matched.drop_duplicates(['aamc_id','entity_id'])
matched = matched.sort_values('grad_yr').drop_duplicates('aamc_id', keep='last')

In [ ]:
# pure = matched.drop_duplicates('aamc_id', keep=False)
# duplicates = matched[matched.aamc_id.isin(pure.aamc_id)==False]

In [ ]:
status = pd.read_excel('../../Data/SRS/Status_Crosswalk.xlsx')
schools = pd.read_excel("../../Data/SRS/US_Schools_Crosswalk_AMA_AAMC.xlsx")

In [ ]:
founded = found_students[found_students['Curr Exp Grad Date']!='None']
unfounded = found_students[found_students['Curr Exp Grad Date']=='None']
founded['CURRENT_GRAD_YEAR'] = [x.year for x in pd.to_datetime(founded['Curr Exp Grad Date'])]
unfounded['CURRENT_GRAD_YEAR'] = 'None'
found_students = pd.concat([founded, unfounded])
grad_match = []
for row in found_students.itertuples():
    if row.CURRENT_GRAD_YEAR != row.grad_yr:
        grad_match.append(False)
    else:
        grad_match.append(True)
found_students['grad_match'] = grad_match
grad_mismatch = found_students[found_students.grad_match == False]
grad_mismatch = pd.merge(grad_mismatch, status, left_on='Curr Status', right_on='STATUS_DESC', how='left')
grad_mismatch = pd.merge(grad_mismatch, schools, on='INST_ID', how='left')

In [ ]:
student_status = pd.merge(found_students, status, left_on='Curr Status', right_on='STATUS_DESC', how='left')
student_status = student_status.rename(columns={'AMA Status':'ama_status'})

In [ ]:
status_match = []
for row in student_status.itertuples():
    if row.ama_status != row.edu_sts:
        status_match.append(False)
    else:
        status_match.append(True)
student_status['status_match'] = status_match
status_mismatch = student_status[student_status.status_match == False]
status_mismatch = pd.merge(status_mismatch, schools, on='INST_ID', how='left')
status_update = status_mismatch[status_mismatch.aamc_id.isin(grad_year.aamc_id)==False]

In [ ]:
not_found[['aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc',
'stud_id',
'school_id',
'edu_sts',
'grad_yr',
'edu_pref_cd',
'degree_cd',
'CURRENT_GRAD_YEAR',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'AMA Status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE',
'INST_NAME',
'AMA_ID']]

In [ ]:
grad_year = grad_mismatch[['aamc_id',
  'stud_id',
  'grad_yr',
  'CURRENT_GRAD_YEAR',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'AMA Status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
#.to_csv('../../Data/SRS/SRS_Graduation_Year_Mismatch_2021-01-13.csv', index=False)

In [ ]:
still_missing = pd.merge(still_missing, schools, on='INST_ID', how='left')

In [ ]:
still_missing = still_missing[['INST_ID',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID','aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
# .to_csv('../../Data/SRS/SRS_Missing_2021-01-13.csv', index=False)

In [ ]:
status_update = status_update[['aamc_id',
  'stud_id',
  'grad_yr',
  'CURRENT_GRAD_YEAR',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'ama_status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
#.to_csv('../../Data/SRS/SRS_Status_Mismatch_2021-01-13.csv', index=False)

In [ ]:
rulz = pd.read_excel('../../Data/SRS/Processing _Rules.xlsx')

In [ ]:
yall = pd.concat([grad_year, status_update]).drop_duplicates('aamc_id')

In [ ]:
stats = pd.merge(rulz, status_update, left_on = ['Curr Status', 'edu_status'], right_on =['Curr Status', 'edu_sts'])

In [ ]:
manual = stats[(stats['PROCESS?']=='N')&(stats['NEW EDU_STS/ACTION']=='Flag for Manual Review')]

In [ ]:
auto = stats[(stats['PROCESS?']=='Y')][['aamc_id','NEW EDU_STS/ACTION','NEW STS REASON','NEW CATEGORY CODE','Curr Status Date','Curr Exp Grad Date']]

In [ ]:
all_other_sts = status_update[(status_update.aamc_id.isin(auto.aamc_id)==False)&(status_update.aamc_id.isin(manual.aamc_id)==False)]

In [ ]:
other_grad = grad_year[(grad_year.aamc_id.isin(auto.aamc_id)==False)&(grad_year.aamc_id.isin(manual.aamc_id)==False)]

In [ ]:
stats[(stats['PROCESS?']=='N')].groupby('NEW EDU_STS/ACTION').count()

In [ ]:
other_sts = status_update[status_update.aamc_id.isin(stats.aamc_id)==False]

In [ ]:
unprocessed = stats[(stats['PROCESS?']=='N')&(stats['NEW EDU_STS/ACTION']!='Flag for Manual Review')]

In [ ]:
len(status_update)

In [ ]:
manual = pd.concat([manual, other_sts])

In [ ]:
# scraped
#found
# matched
# missing
# found_students
# stu_affil
# grad_year
# status_update
# manual
# auto
# new
print(
f'''
{"{:,}".format(len(scraped_new))} student records in AAMC-SRS scrape
{"{:,}".format(len(found_info))} ({round(len(found_info)/len(scraped_new)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped_new)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped_new)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped_new)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual))} status discrepancies flagged for manual review
{"{:,}".format(len(auto))} status discrepancies automatically processed
{"{:,}".format(len(unprocessed))} status discrepancies not processed
{"{:,}".format(len(new))} record updates since last scrape
'''
)

In [ ]:
len(missing)

In [ ]:
93,961 student records in AAMC-SRS scrape
91,435 (97.31%) records matched to AIMS on aamc_id
1,917 (2.04%) MEs appended via matching process
609 (0.65%) scraped records missing from our database

----

Of the records that matched on aamc_id
19,764 (21.62%) records are student affiliates
71,684 (78.4%) records are not student affiliates
    
----
Of the records that are not student affiliates
4,542 (6.34%) records have graduation year discrepancies
4,810 (6.71%) records have status discrepancies, but correct graduation year

----

4,660 status discrepancies flagged for manual review
4,692 records automatically processed
1,046 record updates since last scrape

In [ ]:
pd.read_excel('../../Data/SRS/US_Schools_Crosswalk_AMA_AAMC.xlsx')

In [ ]:
scraped['Curr Status Date'] = pd.to_datetime(scraped['Curr Status Date'])
scraped_new['Curr Status Date'] = pd.to_datetime(scraped_new['Curr Status Date'])

In [ ]:
scraped.columns

In [ ]:
all_scraped = pd.merge(scraped, scraped_new, on='AAMC ID', suffixes = ['_old', ''], how = 'right')

In [ ]:
all_scraped['Time_Diff'] = all_scraped['Curr Status Date'] - all_scraped['Curr Status Date_old']

In [ ]:
new = all_scraped[all_scraped['Time_Diff']!='0 days'][scraped.columns]

In [ ]:
new

In [ ]:
me_matched = pd.merge(matched, me_entity, on='entity_id')

In [ ]:
me_matched

In [ ]:
for col in matched.columns:
    print(col)

In [ ]:
me_matched[['me', 'Date of Birth',
'birth_dt',
'Name',
'first_nm',
'middle_nm',
'last_nm',
'gender',
'Sex', 'Birth State', 'birth_state_cd',
]]

In [ ]:
matched = matched[['me','aamc_id',
'entity_id',
'school_id', 'category_code',
'stud_id', 'Date of Birth',
'birth_dt',
'Name',
'first_nm',
'middle_nm',
'last_nm',
'gender',
'Sex', 'Birth State', 'birth_state_cd',
'grad_yr',
'degree_cd',
'Current Degree Program',
'Curr Class Level',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date'
]]

In [ ]:
matched = pd.merge(matched, me_entity, on='entity_id', how='left')

In [ ]:
# scraped
#found
# matched
# missing
# found_students
# stu_affil
# grad_year
# status_update
# manual
# auto
# new
f'''
{"{:,}".format(len(scraped))} student records in AAMC-SRS scrape
{"{:,}".format(len(found))} ({round(len(found)/len(scraped)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual) + len(all_other_sts))} status discrepancies flagged for manual review
{"{:,}".format(len(auto) + len(other_grad))} records automatically processed
{"{:,}".format(len(new))} record updates since last scrape
'''

In [ ]:
me_matched.columns

In [ ]:
matched.columns

In [ ]:
f'''
{"{:,}".format(len(scraped_new))} student records in AAMC-SRS scrape
{"{:,}".format(len(found))} ({round(len(found)/len(scraped_new)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped_new)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped_new)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped_new)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual) + len(all_other_sts))} status discrepancies flagged for manual review
{"{:,}".format(len(auto) + len(other_grad))} records automatically processed
{"{:,}".format(len(new))} record updates since last scrape
''')

In [ ]:
matched
missing
stu_affil
grad_year
status_discrepancy
new


In [ ]:
grad_year[grad_year.aamc_id.isin(manual.aamc_id)]

In [ ]:
grad_year

In [ ]:
still_missing

In [ ]:
for col in stu_affil.columns:
    print(col)

In [ ]:
stu_affil = stu_affil[['category_code','aamc_id',
  'stud_id',
  'grad_yr',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [ ]:
unprocessed = unprocessed[['aamc_id',
  'stud_id',
  'grad_yr',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [ ]:
new = new[['aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [ ]:
new['aamc_id']=new['AAMC ID']

In [ ]:
news = new.aamc_id

In [ ]:
scraped_new['NEW'] = ['Y' if x in list(new.aamc_id) else 'N' for x in scraped_new.aamc_id]

In [ ]:
[x for x in scraped_new.aamc_id]

In [ ]:
scraped_new[scraped_new.NEW=='Y']

In [ ]:
new

In [ ]:
scraped_new[scraped_new.aamc_id.isin(new.aamc_id)]

In [ ]:
newly = scraped[['aamc_id','NEW']]

In [ ]:
pd.merge(matched, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Matched_2021-03-19.csv', index=False)
pd.merge(still_missing, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Missing_2021-03-19.csv', index=False)
pd.merge(grad_year, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Graduation_Year_Discrepancy_2021-03-19.csv', index=False)
pd.merge(status_update, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Status_Discrepancy_2021-03-19.csv', index=False)
pd.merge(stu_affil, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Student_Affiliates_2021-03-19.csv', index=False)
pd.merge(auto, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Automatic_Updates_2021-03-19.csv', index=False)
pd.merge(manual, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Manual_Updates_2021-03-19.csv', index=False)
pd.merge(unprocessed, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Unprocessed_2021-03-19.csv', index=False)
new.to_csv('../../Data/SRS/SRS_New_2021-03-19.csv', index=False)

In [ ]:
all_students.to_csv('../../Data/SRS/all_students.csv', index=False)

In [ ]:
for col in no_id:
    print(col)

In [ ]:
no_id[['AAMC ID', 'Sex', 'birth_dt', 'birth_state_cd', 'RaceEthnicity', 'Birth County', 'Name', 'last_nm', 'Current Degree Program', 'Curr Exp Grad Date', 'first_middle']].to_csv('../../Data/SRS/no_ids.csv', index=False)